In [1]:
import sys
import os

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("Project root added to path:", PROJECT_ROOT)


Project root added to path: d:\ML projects\player-behavior-sequence-modeling


In [2]:
from src.dataset import PlayerSequenceDataset


In [3]:

sequences_path = os.path.join(PROJECT_ROOT, "data", "processed", "sequences.npy")
masks_path = os.path.join(PROJECT_ROOT, "data", "processed", "masks.npy")

dataset =PlayerSequenceDataset(
    sequences_path,
    masks_path
)

print(len(dataset))
print(dataset[0])


100
{'sequence': tensor([8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'mask': tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}


In [4]:
import torch
from src.model import PlayerBehaviourLSTM

In [5]:
batch_size = 4
batch_sequences = torch.stack(
    [dataset[i]["sequence"] for i in range(batch_size)]
)

In [6]:
print("Batch shape:", batch_sequences.shape)

Batch shape: torch.Size([4, 20])


In [7]:
model = PlayerBehaviourLSTM(
    num_actions=10,
    embedding_dim=32,
    hidden_dim=64
)

In [8]:
output = model(batch_sequences)
print("Output shape:", output.shape)

Output shape: torch.Size([4, 64])


In [9]:
import torch.nn as nn
from torch.utils.data import DataLoader
from src.model import PlayerBehaviorNextAction


In [10]:
dataset = PlayerSequenceDataset(sequences_path, masks_path)

In [11]:
def collate_fn(batch):
    sequences = torch.stack([b["sequence"] for b in batch])
    masks = torch.stack([b["mask"] for b in batch])

    # Inputs: all except last token
    x = sequences[:, :-1]

    # Targets: next action (last real action per sequence)
    # We’ll predict the LAST non-padding action
    y = []
    for seq, m in zip(sequences, masks):
        last_idx = m.sum().item() - 1
        last_idx = max(last_idx, 0)
        y.append(seq[last_idx])
    y = torch.tensor(y, dtype=torch.long)

    return x, y


loader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=collate_fn
)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PlayerBehaviorNextAction(
    num_actions=10,
    embedding_dim=32,
    hidden_dim=64
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)  # ignore padding
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [13]:
epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for x, y in loader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")


Epoch 1/5 - Loss: 2.3388
Epoch 2/5 - Loss: 2.2590
Epoch 3/5 - Loss: 2.1557
Epoch 4/5 - Loss: 1.7802
Epoch 5/5 - Loss: 1.4566
